# Etapa 2 - Integração e Limpeza de Dados

In [6]:
# Bibliotecas

import duckdb
import pandas as pd
import numpy as np

# Carregamento do Dataset
path = '/content/drive/MyDrive/Mortalidade_Geral_2020.csv'
df = pd.read_csv(path, sep=';')

/tmp/ipython-input-4274083977.py:9: DtypeWarning: Columns (66,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=';')


## Limpeza do Dataset

In [45]:
# Filtrando Dataset

# Colunas necessárias para o trabalho
df_filtrado = df[['IDADE', 'DTOBITO', 'CODMUNOCOR', 'SEXO', 'RACACOR', 'LOCOCOR', 'ESCFALAGR1', 'ESC', 'ESC2010', 'ASSISTMED', 'CIRCOBITO', 'NECROPSIA', 'CAUSABAS', 'TPPOS', 'LINHAII']]

# Colunas a serem filtradas

# Normalizar coluna idade
def converte_idade_valor(valor):
    if pd.isna(valor):
        return np.nan  # nulo se não for número

    # Garantir que é positivo (ex: 415.0 -> 415)
    valor = int(valor)

    unidade = valor // 100       # primeiro dígito
    quantidade = valor % 100     # últimos dois dígitos

    if unidade == 1:  # minutos
        return 0
    elif unidade == 2:  # horas
        return 0
    elif unidade == 3:  # meses
        return 0
    elif unidade == 4:  # anos
        return quantidade
    elif unidade == 5:  # +100 anos
        return 100 + quantidade
    else:
        return np.nan  # valor inválido

df_filtrado['IDADE'] = df_filtrado['IDADE'].apply(converte_idade_valor)

# Filtro para apenas top-10 capitais
# São Paulo (SP)             | 3550308     |
# Rio de Janeiro (RJ)        | 3304557     |
# Brasília (DF)              | 5300108     |
# Fortaleza (CE)             | 2304400     |
# Salvador (BA)              | 2927408     |
# Belo Horizonte (MG)        | 3106200     |
# Manaus (AM)                | 1302603     |
# Curitiba (PR)              | 4106902     |
# Recife (PE)                | 2611606     |
# Goiânia (GO)               | 5208707     |
codigos_capitais = [355030, 330455, 530010, 230440, 292740, 310620, 130260, 410690, 261160, 520870]
df_filtrado = df_filtrado[df_filtrado['CODMUNOCOR'].isin(codigos_capitais)]

# Filtro de doenças respiratórias
df_filtrado['CAUSABAS'] = df_filtrado['CAUSABAS'].apply(
    lambda x: 1 if isinstance(x, str) and (
        x.startswith('A15') or
        x.startswith('A16') or
        x.startswith('A17') or
        x.startswith('A18') or
        x.startswith('A19') or
        x.startswith('J') or
        x.startswith('B342')
    ) else 0
)
df_filtrado.rename(columns={'CAUSABAS': 'DOENCA_RESPIRATORIA'}, inplace=True)

df_filtrado['LINHAII'] = df_filtrado['LINHAII'].apply(
    lambda x: 1 if isinstance(x, str) and (
        x.startswith('*A15') or
        x.startswith('*A16') or
        x.startswith('*A17') or
        x.startswith('*A18') or
        x.startswith('*A19') or
        x.startswith('*J') or
        x.startswith('*B342')
    ) else 0
)
df_filtrado.rename(columns={'LINHAII': 'DOENCA_RESPIRATORIA_CONTRIBUINTE'}, inplace=True)

# Peso para necropsia, óbitos com necropsia = 0.75, óbitos sem necropsia = 0.25
df_filtrado = df_filtrado[df_filtrado['NECROPSIA'] != 9]
df_filtrado['NECROPSIA'] = df_filtrado['NECROPSIA'].apply(lambda x: 0.75 if x == 1 else 0.25)

# Peso para óbitos com investigação, óbitos com investigação = 0.75, óbitos sem investigação = 0.25
df_filtrado['TPPOS'] = df_filtrado['TPPOS'].apply(lambda x: 0.75 if x == "S" else 0.25)

# Remoção de tipo de morte violenta ou circunstâncias emque se deu a morte não natural
df_filtrado = df_filtrado[df_filtrado['CIRCOBITO'].isna()]

# Remoção de informação não informadas
df_filtrado = df_filtrado[df_filtrado['ESCFALAGR1'] < 9]
df_filtrado = df_filtrado[(df_filtrado['ESC'] != 9) & (df_filtrado['ESC2010'] != 9)]
df_filtrado = df_filtrado[df_filtrado['LOCOCOR'] < 9]
df_filtrado = df_filtrado[df_filtrado['ASSISTMED'] < 9]


/tmp/ipython-input-3872298602.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['IDADE'] = df_filtrado['IDADE'].apply(converte_idade_valor)


## Conversão dos datasets para Parquet

In [55]:
# Divindo dataset em meses

# Criar coluna com mês de óbito
# Exemplo: 6052020 -> 605 -> 05 -> mês 5
df_filtrado['MES_OBITO'] = ((df_filtrado['DTOBITO'] // 10000) % 100).astype(int)

# Criar um dicionário para armazenar os 12 DataFrames (um por mês)
datasets_mensais = {}

for mes in range(1, 13):
    df_mes = df_filtrado[df_filtrado['MES_OBITO'] == mes].copy()
    datasets_mensais[mes] = df_mes
    # Salvar em Parquet
    nome_arquivo = f"obitos_mes_{mes:02d}.parquet"
    df_mes.to_parquet(nome_arquivo, index=False)
    print(f"Dataset do mês {mes:02d} salvo com {len(df_mes)} linhas.")


Dataset do mês 01 salvo com 3855 linhas.
Dataset do mês 02 salvo com 3862 linhas.
Dataset do mês 03 salvo com 3999 linhas.
Dataset do mês 04 salvo com 5048 linhas.
Dataset do mês 05 salvo com 6220 linhas.
Dataset do mês 06 salvo com 5141 linhas.
Dataset do mês 07 salvo com 5537 linhas.
Dataset do mês 08 salvo com 5517 linhas.
Dataset do mês 09 salvo com 4907 linhas.
Dataset do mês 10 salvo com 4689 linhas.
Dataset do mês 11 salvo com 4742 linhas.
Dataset do mês 12 salvo com 5659 linhas.
